In [1]:
import csv
import numpy as np
import os
import pandas as pd
import math

In [10]:
user_app_usage_appId = pd.read_csv('csv_data/user_app_usage.csv', header=None, usecols=[0])
user_app_usage_appId.info()

KeyboardInterrupt: 

In [8]:
len(set(user_app_usage_appId[0].values))

1992667

In [4]:
len(user_app_usage_appId[1].unique())

488124

In [ ]:
user_app_usage_appId_counts = user_app_usage_appId[1].value_counts()
user_app_usage_appId_sorted = user_app_usage_appId_counts.sort_values(ascending=False)
user_app_usage_appId_top100 = user_app_usage_appId_sorted[0:100]
print(user_app_usage_appId_top100)
user_app_usage_appId_sorted[0:100].plot(kind='line', grid=True, label='S1',title='This is Series', style='--')
user_app_usage_appId_sorted[0:1000].apply(lambda x: math.log10(x)).plot(kind='line', grid=True, label='S1',title='This is Series', style='--')

top100List = list(user_app_usage_appId_top100.index)
print(top100List)

In [2]:
top100List = ['a00289791', 'a00103666', 'a00289826', 'a00287085', 'a00278905', 'a0048562', 'a00289850', 'a0046242', 'a00170432', 'a00278858', 'a00290015', 'a0030644', 'a00247519', 'a00271087', 'a0048332', 'a00109386', 'a00289866', 'a00292780', 'a00224427', 'a00290027', 'a00187480', 'a00261220', 'a0021880', 'a00274701', 'a00348507', 'a00292472', 'a00336224', 'a00289511', 'a0022276', 'a0071100', 'a00311680', 'a00481059', 'a0036392', 'a00275200', 'a00278849', 'a00157201', 'a00244790', 'a00263057', 'a00289519', 'a00331160', 'a00287057', 'a0048522', 'a009932', 'a00307756', 'a0048467', 'a00217423', 'a00363352', 'a00166554', 'a00299200', 'a0027551', 'a003925', 'a0092898', 'a0064339', 'a00144187', 'a0048637', 'a00461814', 'a00271777', 'a00373197', 'a00157220', 'a00289728', 'a00141402', 'a0071095', 'a00276196', 'a00290038', 'a0082383', 'a00324307', 'a00299316', 'a0099559', 'a00336168', 'a00281804', 'a0032847', 'a00169965', 'a0075197', 'a00294247', 'a00331594', 'a00278867', 'a00278912', 'a00160089', 'a00261226', 'a00219676', 'a0049622', 'a0082007', 'a00239261', 'a0048452', 'a00348136', 'a00248909', 'a00287071', 'a00135785', 'a00235015', 'a00287090', 'a00101827', 'a00289466', 'a00225310', 'a00153189', 'a0029508', 'a0017114', 'a00294240', 'a00158371', 'a00290067', 'a00225081']

In [3]:
weekends = ['2019-03-02','2019-03-03','2019-03-09','2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24']
user_app_usage = pd.read_csv('csv_data/user_app_usage.csv',header=None,chunksize=5000000)
user_app_usage.columns = ['uId','appId','duration','times','use_date']

In [13]:
for each in user_app_usage:
    break
each.columns = ['uId','appId','duration','times','use_date']
each.head()

,uId,appId,duration,times,use_date
0,1000364,a00289850,752.0,2.0,2019-03-01
1,1000364,a00287085,836.0,19.0,2019-03-12
2,1000364,a00289826,166.0,23.0,2019-03-01
3,1000364,a00287085,836.0,19.0,2019-03-13
4,1000364,a00278858,123.0,45.0,2019-03-19


In [4]:
def merge_by_date(each):
    def f(df):
        duration = df['duration']
        app = df['appId']
        times = df['times']
        date = df['use_date']
        weekdayDuration = [ 0 for x in range(100) ]
        weekdayTimes = [ 0 for x in range(100) ]
        weekendDuration = [ 0 for x in range(100) ]
        weekendTimes = [ 0 for x in range(100) ]        
        for a,t,du,da in zip(app,times,duration,date):
            if a in top100List:
                index = top100List.index(a)
                if da in weekends:
                    weekendTimes[index] += t
                    weekendDuration[index] += du
                else:
                    weekdayTimes[index] += t
                    weekdayDuration[index] += du                
    #     df = pd.Series(data)
        weekD =  sum(weekdayDuration)
        weekendD = sum(weekendDuration)
        weekT = sum(weekdayTimes)
        weekendT = sum(weekendTimes)
        if( weekD==0 or weekendD==0 or weekT==0 or weekendT==0 ):
            return [0]
        rate1 = weekD/weekendD
        rate2 = weekT/weekendT
        return weekdayDuration + weekendDuration + [weekD]+[weekendD]+[rate1] + weekdayTimes + weekendTimes + [weekT]+[weekendT]+[rate2]
    temp = each.groupby(['uId']).apply(f)
    #temp.index.names = ['']
    #df_temp = temp.unstack()
    return temp

In [5]:
cols = ['uId']
cols += ['weekdayDuration_'+str(i) for i in range(100)]+['weekendDuration_'+str(i) for i in range(100)]
cols += ['weekdayDurationSum']+['weekendDurationSum']+['rate1']
cols+= ['weekdayTimes_'+str(i) for i in range(100)] + ['weekendTimes_'+str(i) for i in range(100)]
cols += ['weekdayTimesSum']+['weekendTimesSum']+['rate2']
print(len(cols))

407


In [6]:
RESULT = []
for each in user_app_usage:
    each.columns = ['uId','appId','duration','times','use_date']
    new_one = merge_by_date(each)    
    #print(new_one)
    #print(type(new_one))    
#     new_one.drop(list(new_one.index)[0], axis=0, inplace=True)
#     new_one.drop(list(new_one.index)[-1], axis=0, inplace=True)
    temp = pd.DataFrame(list(new_one.values))
    temp.index = new_one.index
    temp.index.name = '' 
    RESULT.append(temp)
#     break

for each in RESULT:
    each.columns = cols[1:]
user_app_usage_features = pd.concat(RESULT)
# user_app_usage_features = user_app_usage_features.dropna()

In [7]:
user_app_usage_features.head()

,weekdayDuration_0,weekdayDuration_1,weekdayDuration_2,weekdayDuration_3,weekdayDuration_4,weekdayDuration_5,weekdayDuration_6,weekdayDuration_7,weekdayDuration_8,weekdayDuration_9,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
,,,,,,,,,,,,,,,,,,,,,
1000107,15854.0,8821.0,310.0,6957.0,0.0,0.0,0.0,3541.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10602.0,5012.0,2.115323
1000321,37228.0,4176.0,16616.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,86.0,0.0,0.0,0.0,0.0,0.0,12518.0,3627.0,3.451337
1000364,137527.0,0.0,40509.0,32381.0,0.0,0.0,14319.0,0.0,0.0,15826.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4598.0,1786.0,2.574468
1000387,93777.0,5096.0,2106.0,33037.0,0.0,12515.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19491.0,8782.0,2.219426
1000395,35678.0,836.0,2855.0,360.0,0.0,0.0,0.0,0.0,3411.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5423.0,3043.0,1.782123


In [12]:
import pickle
for i in range(len(RESULT)):
    with open('user_app_usage_features_na/user_app_usage_features'+str(i)+'.pkl','wb') as f:
        pickle.dump(RESULT[i],f)

In [13]:
user_app_usage_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1977442 entries, 1000237 to 3511958
Columns: 406 entries, weekdayDuration_0 to rate2
dtypes: float64(406)
memory usage: 6.0 GB


In [14]:
user_app_usage_features.head()

,weekdayDuration_0,weekdayDuration_1,weekdayDuration_2,weekdayDuration_3,weekdayDuration_4,weekdayDuration_5,weekdayDuration_6,weekdayDuration_7,weekdayDuration_8,weekdayDuration_9,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
,,,,,,,,,,,,,,,,,,,,,
1000237,58033.0,2447.0,1258.0,61578.0,12067.0,5297.0,0.0,0.0,465.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12772.0,5055.0,2.526607
1000242,3227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720.0,15.0,48.000000
1000754,26531.0,5330.0,0.0,64830.0,57131.0,1546.0,0.0,2909.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15512.0,5887.0,2.634958
1001019,183213.0,6844.0,6996.0,4248.0,10567.0,0.0,0.0,842.0,2220.0,49653.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26860.0,10240.0,2.623047
1001253,153505.0,6917.0,183.0,15786.0,27064.0,687.0,23626.0,25357.0,0.0,0.0,...,0.0,0.0,96.0,0.0,0.0,0.0,36.0,18654.0,5493.0,3.395958


In [12]:
import re
re.split(r'[eE]','1233e43E43')

['1233', '43', '43']